# BayScen Scenario Generation - Tutorial

This notebook demonstrates how to generate test scenarios using the BayScen framework.

## Overview

The generation process:
1. Load trained Bayesian Network model
2. Generate scenarios using combinatorial coverage + conditional sampling
3. Evaluate scenarios (realism, coverage, diversity)
4. Export for testing

**Scenarios:**
- Scenario 1: Vehicle-Vehicle conflicts (8 environmental variables)
- Scenario 2: Vehicle-Cyclist conflicts (9 environmental variables + Time_of_Day)

**Modes:**
- `rare`: Prioritize edge cases (recommended for testing)
- `common`: Prioritize typical scenarios

## Setup

In [1]:
import sys
import pickle
import pandas as pd
from pathlib import Path

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

from generation.scenario_generator import BayesianScenarioGenerator
from generation.evaluation_metrics import (
    evaluate_scenarios,
    compute_realism,
    compute_threeway_coverage
)
from generation.generation_utils import validate_scenarios, split_by_collision_point
from abstraction.abstract_variables import LEAF_NODES

import warnings
warnings.filterwarnings('ignore')

## Configuration

In [2]:
# Choose scenario and mode
SCENARIO = 2  # 1 or 2
MODE = 'rare'  # 'common' or 'rare'

# Define paths
MODEL_PATH = f"../modeling/models/scenario{SCENARIO}_full_bayesian_network.pkl"
DATA_PATH = "../../data/processed/bayscen_final_data.csv"
OUTPUT_DIR = Path("generated_scenarios")
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"Configuration:")
print(f"  Scenario: {SCENARIO}")
print(f"  Mode: {MODE}")
print(f"  Model: {MODEL_PATH}")
print(f"  Output: {OUTPUT_DIR}")

Configuration:
  Scenario: 2
  Mode: rare
  Model: ../modeling/models/scenario2_full_bayesian_network.pkl
  Output: generated_scenarios


## Step 1: Load Trained Model

In [3]:
# Load the Bayesian Network
print("Loading Bayesian Network...")
with open(MODEL_PATH, 'rb') as f:
    model = pickle.load(f)

print(f"✓ Model loaded successfully")
print(f"  Total nodes: {len(model.nodes())}")
print(f"  Total edges: {len(model.edges())}")

Loading Bayesian Network...
✓ Model loaded successfully
  Total nodes: 17
  Total edges: 24


## Step 2: Define Variables

In [4]:
# Abstracted variables (from abstract_variables.py)
abstracted_variables = LEAF_NODES
print("Abstracted Variables:")
for var, values in abstracted_variables.items():
    print(f"  {var}: {values}")

# Concrete variables (scenario-specific)
concrete_variables_s1 = [
    "Cloudiness", "Wind_Intensity", "Precipitation", "Precipitation_Deposits",
    "Wetness", "Fog_Density", "Road_Friction", "Fog_Distance",
    "Start_Ego", "Goal_Ego", "Start_Other", "Goal_Other"
]

concrete_variables_s2 = ["Time_of_Day"] + concrete_variables_s1

concrete_variables = concrete_variables_s2 if SCENARIO == 2 else concrete_variables_s1

print(f"\nConcrete Variables ({len(concrete_variables)}): {concrete_variables}")

Abstracted Variables:
  Collision_Point: ['c1', 'c2', 'c3']
  Visibility: [0, 20, 40, 60, 80, 100]
  Road_Surface: [0, 20, 40, 60, 80, 100]
  Vehicle_Stability: [0, 20, 40, 60, 80, 100]

Concrete Variables (13): ['Time_of_Day', 'Cloudiness', 'Wind_Intensity', 'Precipitation', 'Precipitation_Deposits', 'Wetness', 'Fog_Density', 'Road_Friction', 'Fog_Distance', 'Start_Ego', 'Goal_Ego', 'Start_Other', 'Goal_Other']


## Step 3: Create Generator

In [5]:
# Create generator
prefer_rare = (MODE == 'rare')

generator = BayesianScenarioGenerator(
    model=model,
    leaf_nodes=abstracted_variables,
    initial_nodes=concrete_variables,
    similarity_threshold=0.1,
    n_samples=100000,
    use_sampling=True,
    prefer_rare=prefer_rare
)

print(f"✓ Generator created (Mode: {MODE}, Prefer rare: {prefer_rare})")

✓ Generator created (Mode: rare, Prefer rare: True)


## Step 4: Generate Scenarios

This will generate all combinations of abstracted variables and sample concrete parameters.

**Expected output:** 648 scenarios (6×6×6×3 combinations)

**Time:** ~90–120 minutes (with n_samples = 100000, depending on your machine) 

In [6]:
# Generate scenarios
scenarios = generator.generate_scenarios()

print(f"\n{'='*70}")
print(f"Generation Summary:")
print(f"  Total scenarios: {len(scenarios)}")
print(f"  Columns: {list(scenarios.columns)}")
print(f"{'='*70}")

Generated 648 abstracted variable combinations

BAYSCEN SCENARIO GENERATION
Mode: RARE (Edge Cases)
Total combinations: 648
Diversity threshold: 0.1
Start time: 2025-12-11 09:57:37



Generating scenarios:   0%|                                                                 | 0/648 [00:00<?, ?combo/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/17 [00:00<?, ?it/s]

Generating scenarios: 100%|██████████████████████████████████████████████████████▉| 647/648 [01:02<00:00,  5.91combo/s]

  0%|          | 0/17 [00:00<?, ?it/s]

Generating scenarios: 100%|███████████████████████████████████████████████████████| 648/648 [01:02<00:00, 10.30combo/s]


GENERATION COMPLETE
End time: 2025-12-11 09:58:39
Elapsed time: 0:01:02.929906
Scenarios generated: 648
Average time per scenario: 0.10s

Assigning T-junction paths based on Collision_Point...
✓ T-junction paths assigned


Generation Summary:
  Total scenarios: 648
  Columns: ['Time_of_Day', 'Cloudiness', 'Wind_Intensity', 'Precipitation', 'Precipitation_Deposits', 'Wetness', 'Fog_Density', 'Road_Friction', 'Fog_Distance', 'Start_Ego', 'Goal_Ego', 'Start_Other', 'Goal_Other', 'Collision_Point', 'Visibility', 'Road_Surface', 'Vehicle_Stability', 'probability']


## Step 5: Inspect Generated Scenarios

In [7]:
# Display first few scenarios
print("First 5 scenarios:")
scenarios.head()

First 5 scenarios:


,Time_of_Day,Cloudiness,Wind_Intensity,Precipitation,Precipitation_Deposits,Wetness,Fog_Density,Road_Friction,Fog_Distance,Start_Ego,Goal_Ego,Start_Other,Goal_Other,Collision_Point,Visibility,Road_Surface,Vehicle_Stability,probability
0,-90,20,80,0,0,60,0,0.8,100,Left,Right,Base,Left,c1,0,0,0,0.1
1,90,60,60,0,0,0,0,0.8,100,Left,Right,Base,Right,c1,0,0,20,0.1
2,-90,20,80,0,0,0,80,0.8,20,Base,Left,Left,Right,c1,0,0,40,0.1
3,0,60,40,60,100,40,40,0.6,60,Base,Right,Left,Right,c1,0,0,60,0.1
4,90,20,80,40,20,60,40,0.8,60,Left,Right,Base,Left,c1,0,0,80,0.1


In [8]:
# Distribution by collision point
print("\nScenarios by Collision Point:")
print(scenarios['Collision_Point'].value_counts())

# Split by collision point
by_collision = split_by_collision_point(scenarios)
for cp, df in by_collision.items():
    print(f"  {cp}: {len(df)} scenarios")


Scenarios by Collision Point:
Collision_Point
c1    216
c2    216
c3    216
Name: count, dtype: int64
  c1: 216 scenarios
  c2: 216 scenarios
  c3: 216 scenarios


## Step 6: Validate Scenarios

In [9]:
# Validate scenarios
validation = validate_scenarios(scenarios)

print("Validation Results:")
print(f"  Valid: {validation['is_valid']}")
print(f"  Total scenarios: {validation['num_scenarios']}")

if not validation['is_valid']:
    print(f"\n  Issues found:")
    for issue in validation['issues']:
        print(f"    - {issue}")
else:
    print("  ✓ All scenarios are valid!")

Validation Results:
  Valid: True
  Total scenarios: 648
  ✓ All scenarios are valid!


## Step 7: Evaluate Scenarios

Compare generated scenarios against real-world data.

In [10]:
# Define evaluation attributes
eval_attributes = [
    "Cloudiness", "Wind_Intensity", "Precipitation", "Precipitation_Deposits",
    "Wetness", "Fog_Density", "Road_Friction", "Fog_Distance"
]

if SCENARIO == 2:
    eval_attributes = ["Time_of_Day"] + eval_attributes

# Define parameter domains for 3-way coverage
parameter_domains = {
    'Cloudiness': [0, 20, 40, 60, 80, 100],
    'Wind_Intensity': [0, 20, 40, 60, 80, 100],
    'Precipitation': [0, 20, 40, 60, 80, 100],
    'Precipitation_Deposits': [0, 20, 40, 60, 80, 100],
    'Wetness': [0, 20, 40, 60, 80, 100],
    'Road_Friction': [0.1, 0.2, 0.4, 0.6, 0.8, 1.0],
    'Fog_Density': [0, 20, 40, 60, 80, 100],
    'Fog_Distance': [0, 20, 40, 60, 80, 100]
}

if SCENARIO == 2:
    parameter_domains['Time_of_Day'] = [-90, -60, -30, 0, 30, 60, 90]

# Evaluate
results = evaluate_scenarios(
    generated_df=scenarios,
    real_data_path=DATA_PATH,
    attributes=eval_attributes,
    parameter_domains=parameter_domains,
    print_summary=True
)


SCENARIO EVALUATION RESULTS

1. REALISM
   Overall: 99.07%
   (99.07% of scenarios within real-world distribution)

2. 3-WAY COVERAGE
   Triples covered: 3910
   All possible: 32.32%
   Real-world: 62.41%
   Precision: 97.37%
   Recall: 62.41%
   F1 Score: 76.06



### Detailed Metrics

In [11]:
# Realism metric
print(f"\nRealism: {results['realism']:.2f}%")
print(f"  → {results['realism']:.2f}% of scenarios are within real-world distribution")

if results['realism'] > 90:
    print("  ✓ Excellent realism!")
elif results['realism'] > 70:
    print("  ✓ Good realism")
else:
    print("  ⚠ Consider adjusting generation parameters")


Realism: 99.07%
  → 99.07% of scenarios are within real-world distribution
  ✓ Excellent realism!


In [12]:
# 3-Way Coverage
coverage = results['coverage_3way']
print(f"\n3-Way Coverage:")
print(f"  Triples covered: {coverage['triples_covered']}")
print(f"  All possible: {coverage['all_triples_pct']:.2f}%")
print(f"  Real-world: {coverage['real_triples_pct']:.2f}%")
print(f"  Precision: {coverage['precision']:.2f}%")
print(f"  Recall: {coverage['recall']:.2f}%")
print(f"  F1 Score: {coverage['f1_score']:.2f}")

if coverage['f1_score'] > 85:
    print("  ✓ Excellent coverage!")
elif coverage['f1_score'] > 70:
    print("  ✓ Good coverage")


3-Way Coverage:
  Triples covered: 3910
  All possible: 32.32%
  Real-world: 62.41%
  Precision: 97.37%
  Recall: 62.41%
  F1 Score: 76.06
  ✓ Good coverage


## Step 8: Save Scenarios

In [13]:
# Save to CSV
output_csv = OUTPUT_DIR / f"scenario{SCENARIO}_{MODE}_scenarios.csv"
scenarios.to_csv(output_csv, index=False)
print(f"✓ Saved to: {output_csv}")

# Save evaluation results
eval_output = OUTPUT_DIR / f"scenario{SCENARIO}_{MODE}_evaluation.pkl"
with open(eval_output, 'wb') as f:
    pickle.dump(results, f)
print(f"✓ Saved evaluation to: {eval_output}")

✓ Saved to: generated_scenarios\scenario2_rare_scenarios.csv
✓ Saved evaluation to: generated_scenarios\scenario2_rare_evaluation.pkl


## Comparison: Common vs Rare Modes (Optional)

If you've generated both modes, you can compare them:

In [ ]:
# Load both modes for comparison
try:
    common_scenarios = pd.read_csv(OUTPUT_DIR / f"scenario{SCENARIO}_common_scenarios.csv")
    rare_scenarios = pd.read_csv(OUTPUT_DIR / f"scenario{SCENARIO}_rare_scenarios.csv")
    
    print("Comparing Common vs Rare Modes:\n")
    
    print(f"Common Scenarios:")
    print(f"  Count: {len(common_scenarios)}")
    print(f"  Mean probability: {common_scenarios['probability'].mean():.6f}")
    print(f"  Median probability: {common_scenarios['probability'].median():.6f}")
    
    print(f"\nRare Scenarios:")
    print(f"  Count: {len(rare_scenarios)}")
    print(f"  Mean probability: {rare_scenarios['probability'].mean():.6f}")
    print(f"  Median probability: {rare_scenarios['probability'].median():.6f}")
    
    # Compare using utility function
    from generation.generation_utils import compare_scenario_sets
    compare_scenario_sets(
        common_scenarios,
        rare_scenarios,
        eval_attributes,
        "Common",
        "Rare"
    )
    
except FileNotFoundError:
    print("Both common and rare scenarios needed for comparison.")
    print("Generate both modes to compare:")
    print("  MODE = 'common'  # Run once")
    print("  MODE = 'rare'    # Run again")

## Summary

This notebook demonstrated:
1. ✓ Loading trained Bayesian Network
2. ✓ Configuring scenario generator
3. ✓ Generating test scenarios
4. ✓ Validating scenario quality
5. ✓ Evaluating realism, coverage, and diversity
6. ✓ Exporting scenarios for testing

**Next Steps:**
- Run scenarios in CARLA simulator
- Analyze test results
- Compare with baseline methods (Random, SitCov, CTBC, PICT)

## Quick Reference

### Generate Both Scenarios

```python
# Scenario 1 - Rare
SCENARIO = 1
MODE = 'rare'
# Run cells above

# Scenario 1 - Common
SCENARIO = 1
MODE = 'common'
# Run cells above

# Scenario 2 - Rare
SCENARIO = 2
MODE = 'rare'
# Run cells above

# Scenario 2 - Common
SCENARIO = 2
MODE = 'common'
# Run cells above
```

### Command Line Alternative

```bash
cd bayscen/generation

# Generate all variants
python generate_scenarios.py --scenario 1 --mode rare
python generate_scenarios.py --scenario 1 --mode common
python generate_scenarios.py --scenario 2 --mode rare
python generate_scenarios.py --scenario 2 --mode common
```